In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\ajitp\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
sales = pd.read_csv('superstore_dataset2011-2015.csv', encoding='iso-8859-1')

In [4]:
sales.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,42433,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,...,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140,35.46,Medium
1,22253,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium


#### Total Sales Amount

In [5]:
sum(sales['Sales'])

12642501.909880234

In [6]:
sales['Sales'].sum()

12642501.90988

#### Total Quantity

In [7]:
sales['Quantity'].sum()

178312

#### Total Profit

In [8]:
sales['Profit'].sum()

1467457.2912800002

#### Total Shipping Cost

In [9]:
sales['Shipping Cost'].sum()

1352820.69

#### Profit Percentage

In [10]:
# Profit Percentage

1467457/12642501

0.11607331492400119

In [15]:
profitPercentage = (sales['Profit'].sum() / sales['Sales'].sum()) * 100

In [17]:
print ("Profit Percentage: ", round(profitPercentage,2), "%")

Profit Percentage:  11.61 %


In [ ]:
# Inference from Profit Percentage

# For the sales of each $100
# You get profit of $11.61

#### What is the share of each product category in the total sales till date?

In [25]:
(sales.groupby(['Category'])['Sales'].sum() / sales['Sales'].sum()) * 100

Category
Furniture          32.516303
Office Supplies    29.955069
Technology         37.528628
Name: Sales, dtype: float64

In [ ]:
# 1. Technology has the highest sales share followed Furniture and Office Supplies respectively.

#### What is the share of each product category in the total PROFIT till date?

In [26]:
(sales.groupby(['Category'])['Profit'].sum() / sales['Profit'].sum()) * 100

Category
Furniture          19.435300
Office Supplies    35.331443
Technology         45.233257
Name: Profit, dtype: float64

In [27]:
sales.groupby(['Category'])['Profit'].sum()

Category
Furniture          285204.72380
Office Supplies    518473.83430
Technology         663778.73318
Name: Profit, dtype: float64

In [ ]:
# 1. Technology has the highest profit share followed Office Supplies and Furniture respectively.
# 2. Office Supplies is producing nearly $518k worth profit, which is more than 40% of profit from furniture ($285k)

In [29]:
# Explaination: How Profit from Office Supplies in more than 40% from profit from Furniture.

1 - (285/518) # You check the percentage difference
              # You can also multiply this with 100 to convert from ratio to percentage.

0.4498069498069498

#### Annual Sales, and Year over Year growth
#### (Total Sales)

In [31]:
def extractYear(date_value):
    # This function take the date as input
    # and split the date by delimiting character.
    # In this data, we have two types of date formats.
    # Type A: MM/DD/YYYY
    # Type B: DD-MM-YYYY
    if date_value.find('-') == -1: #Type A
        my_date = date_value.split('/')
        year_value = my_date[2]
    else: #Type B
        my_date = date_value.split('-')
        year_value = my_date[2]
    return year_value

In [32]:
years = []
for dd in sales['Order Date']:
    years_value = extractYear(dd)
    years.append(years_value)

In [33]:
sales['year'] = years

In [55]:
sales.groupby(['year'])['Sales'].sum().apply(lambda x: '%.5f' % x) #lambda inline function

year
2011    2259450.89554
2012    2677438.69440
2013    3405746.44938
2014    4299865.87056
Name: Sales, dtype: object

In [62]:
sales_yoy = sales.groupby(['year'])['Sales'].sum()
sales_yoy

year
2011    2.259451e+06
2012    2.677439e+06
2013    3.405746e+06
2014    4.299866e+06
Name: Sales, dtype: float64

In [ ]:
# exponential form

# 100 = 1 x 10^2
# 200 = 2 x 10^2
# 200,000 = 2 x 10^5
# 0.000002 = 2 x 10^-5

# Exponent to number conversion

# 2.259451 e +06   = 2.26 x 10^6 = 2260000

In [49]:
(2677438.69440 - 2259450.89554) / 2677438.69440

0.15611479722551364

In [50]:
1 - (2259450.89554 / 2677438.69440)

0.1561147972255137

In [58]:
pct_change = []

for idx in range(0, len(sales_yoy)):
    if idx == 0:
        pct = 0
    else: 
        pct = 1 - ( sales_yoy[idx-1] / sales_yoy[idx])
    
    pct_change.append(pct)

In [59]:
pct_change

[0, 0.15611479722550914, 0.2138467339847323, 0.20794123540033982]

In [64]:
sales_yoy = pd.DataFrame(sales_yoy) # Remeber that a series can contain only one column
                                    # So you always need to convert a series to dataframe before adding new column to it

In [65]:
sales_yoy['pct_change'] = pct_change

In [66]:
sales_yoy

,Sales,pct_change
year,,
2011,2.259451e+06,0.000000
2012,2.677439e+06,0.156115
2013,3.405746e+06,0.213847
2014,4.299866e+06,0.207941


In [ ]:
# 1. Sales is constantly growing year over year for last three years.
# 2. Sales is growing by 15-20% approx YoY on an average.
# 3. Sales growth has marginally decreased in 2014 in comparison to 2013.

#### Annual Sales, and Year over Year growth
#### (Category-wise Sales)

In [68]:
sales.groupby(['year', 'Category'])['Sales'].sum().apply(lambda x: '%.5f' % x) #lambda inline function

year  Category       
2011  Furniture           756192.38400
      Office Supplies     675606.45020
      Technology          827652.06134
2012  Furniture           858902.55810
      Office Supplies     795094.62700
      Technology         1023441.50930
2013  Furniture          1117723.55340
      Office Supplies    1010717.64190
      Technology         1277305.25408
2014  Furniture          1378055.69080
      Office Supplies    1305651.50640
      Technology         1616158.67336
Name: Sales, dtype: object

In [69]:
sales_yoy_catw = sales.groupby(['year', 'Category'])['Sales'].sum()

In [86]:
sales_yoy_catw = pd.DataFrame(sales_yoy_catw)

In [87]:
sales_yoy_catw.reset_index(inplace=True)

In [88]:
sales_yoy_catw

,year,Category,Sales
0,2011,Furniture,7.561924e+05
1,2011,Office Supplies,6.756065e+05
2,2011,Technology,8.276521e+05
3,2012,Furniture,8.589026e+05
4,2012,Office Supplies,7.950946e+05
5,2012,Technology,1.023442e+06
6,2013,Furniture,1.117724e+06
7,2013,Office Supplies,1.010718e+06
8,2013,Technology,1.277305e+06
9,2014,Furniture,1.378056e+06


In [104]:
sales_yoy_catw.loc[sales_yoy_catw['Category'] == 'Furniture'].to_excel("Furniture_yoy.xls")

In [105]:
sales_yoy_catw.loc[sales_yoy_catw['Category'] == 'Office Supplies'].to_excel("Office_Supplies_yoy.xls")

In [106]:
sales_yoy_catw.loc[sales_yoy_catw['Category'] == 'Technology'].to_excel("Technology_yoy.xls")

#### What quantity of each product is sold till date?

In [108]:
sales.groupby(['Product Name'])['Quantity'].sum()

Product Name
"While you Were Out" Message Book, One Form per Page                                                    8
#10 Gummed Flap White Envelopes, 100/Box                                                               11
#10 Self-Seal White Envelopes                                                                          10
#10 White Business Envelopes,4 1/8 x 9 1/2                                                             32
#10- 4 1/8" x 9 1/2" Recycled Envelopes                                                                37
                                                                                                       ..
iKross Bluetooth Portable Keyboard + Cell Phone Stand Holder + Brush for Apple iPhone 5S 5C 5, 4S 4    24
iOttie HLCRIO102 Car Mount                                                                             12
iOttie XL Car Mount                                                                                    14
invisibleSHIELD by ZAGG Smudge-Fr

In [111]:
product_qty_sold = sales.groupby(['Category','Sub-Category','Product Name'])['Quantity'].sum()

In [115]:
product_qty_sold = pd.DataFrame(product_qty_sold)

In [117]:
product_qty_sold.reset_index(inplace=True)

In [126]:
product_qty_sold

,index,Category,Sub-Category,Product Name,Quantity
0,0,Furniture,Bookcases,"Atlantic Metals Mobile 2-Shelf Bookcases, Cust...",3
1,1,Furniture,Bookcases,"Atlantic Metals Mobile 3-Shelf Bookcases, Cust...",35
2,2,Furniture,Bookcases,"Atlantic Metals Mobile 4-Shelf Bookcases, Cust...",27
3,3,Furniture,Bookcases,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",26
4,4,Furniture,Bookcases,Bestar Classic Bookcase,31
...,...,...,...,...,...
3792,3792,Technology,Phones,iKross Bluetooth Portable Keyboard + Cell Phon...,24
3793,3793,Technology,Phones,iOttie HLCRIO102 Car Mount,12
3794,3794,Technology,Phones,iOttie XL Car Mount,14
3795,3795,Technology,Phones,invisibleSHIELD by ZAGG Smudge-Free Screen Pro...,29


In [118]:
product_qty_sold.to_excel("Product Quantity Sold.xls")

In [129]:
type(product_qty_sold)

pandas.core.frame.DataFrame

In [153]:
selector = []
for pname in product_qty_sold['Product Name']:
    bool_var = pname in top10_products_list
    selector.append(bool_var)

In [155]:
product_qty_sold[selector]

,index,Category,Sub-Category,Product Name,Quantity
216,216,Furniture,Chairs,"Harbour Creations Executive Leather Armchair, ...",142
251,251,Furniture,Chairs,"Hon Executive Leather Armchair, Adjustable",169
322,322,Furniture,Chairs,"Office Star Executive Leather Armchair, Adjust...",141
3238,3238,Technology,Copiers,Canon imageCLASS 2200 Advanced Copier,20
3529,3529,Technology,Phones,"Apple Smart Phone, Full Size",171
3577,3577,Technology,Phones,"Cisco Smart Phone, Full Size",139
3656,3656,Technology,Phones,"Motorola Smart Phone, Full Size",134
3682,3682,Technology,Phones,"Nokia Smart Phone, Full Size",147
3683,3683,Technology,Phones,"Nokia Smart Phone, with Caller ID",96
3764,3764,Technology,Phones,"Samsung Smart Phone, Cordless",108


#### Top 10 Products

In [125]:
top10_products = sales.groupby(['Product Name'])['Sales'].sum().sort_values(ascending=False).head(10)

In [131]:
top10_products

Product Name
Apple Smart Phone, Full Size                                86935.7786
Cisco Smart Phone, Full Size                                76441.5306
Motorola Smart Phone, Full Size                             73156.3030
Nokia Smart Phone, Full Size                                71904.5555
Canon imageCLASS 2200 Advanced Copier                       61599.8240
Hon Executive Leather Armchair, Adjustable                  58193.4841
Office Star Executive Leather Armchair, Adjustable          50661.6840
Harbour Creations Executive Leather Armchair, Adjustable    50121.5160
Samsung Smart Phone, Cordless                               48653.4600
Nokia Smart Phone, with Caller ID                           47877.7857
Name: Sales, dtype: float64

In [132]:
top10_products_list = sales.groupby(['Product Name'])['Sales'].sum().sort_values(ascending=False).head(10).index

In [133]:
top10_products_list

Index(['Apple Smart Phone, Full Size', 'Cisco Smart Phone, Full Size',
       'Motorola Smart Phone, Full Size', 'Nokia Smart Phone, Full Size',
       'Canon imageCLASS 2200 Advanced Copier',
       'Hon Executive Leather Armchair, Adjustable',
       'Office Star Executive Leather Armchair, Adjustable',
       'Harbour Creations Executive Leather Armchair, Adjustable',
       'Samsung Smart Phone, Cordless', 'Nokia Smart Phone, with Caller ID'],
      dtype='object', name='Product Name')

In [135]:
type(top10_products_list)

pandas.core.indexes.base.Index

In [136]:
top10_products_list = list(top10_products_list)
type(top10_products_list)

list

In [138]:
top10_products_list

['Apple Smart Phone, Full Size',
 'Cisco Smart Phone, Full Size',
 'Motorola Smart Phone, Full Size',
 'Nokia Smart Phone, Full Size',
 'Canon imageCLASS 2200 Advanced Copier',
 'Hon Executive Leather Armchair, Adjustable',
 'Office Star Executive Leather Armchair, Adjustable',
 'Harbour Creations Executive Leather Armchair, Adjustable',
 'Samsung Smart Phone, Cordless',
 'Nokia Smart Phone, with Caller ID']

In [124]:
sales.groupby(['Product Name'])['Sales'].sum().sort_values(ascending=False).head(10).values

array([86935.7786, 76441.5306, 73156.303 , 71904.5555, 61599.824 ,
       58193.4841, 50661.684 , 50121.516 , 48653.46  , 47877.7857])

In [165]:
# Exercise: Create total sales for report for product hierarchy
# Exercise: Create total sales for Top 10 Products for City, States, Country, Region

In [ ]:
# Exercise hinting
region_total_sales = sales.groupby(['City','Product Name'])['Sales'].sum()

#### Region wise Sales

In [156]:
region_total_sales = sales.groupby(['Region','Country','State','City','Product Name'])['Sales'].sum()

In [160]:
region_total_sales

Region  Country        State       City       Product Name                                  
Africa  Algeria        Alger       Algiers    Advantus Paper Clips, Metal                         53.520
                                              Apple Headset, with Caller ID                       73.260
                                              Avery 3-Hole Punch, Durable                         29.370
                                              Avery Binder Covers, Recycled                       22.860
                                              Avery Round Labels, Adjustable                       5.040
                                                                                                  ...   
West    United States  Washington  Vancouver  AT&T TR1909W                                       302.376
                                              Boston 16801 Nautilus Battery Pencil Sharpener      44.020
                                              First Data FD10 PIN P

In [161]:
region_total_sales = pd.DataFrame(region_total_sales)
region_total_sales.reset_index(inplace=True)

In [162]:
type(region_total_sales)

pandas.core.frame.DataFrame

In [163]:
selector = []
for pname in region_total_sales['Product Name']:
    bool_var = pname in top10_products_list
    selector.append(bool_var)

In [166]:
region_total_sales[selector].to_excel("region_Total_sales_top10_products.xls")

In [ ]:
# You should focus on creating the tables: How to create different kinds of tables?
# and how to interpret your findings in those tables?